In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd
import numpy as np

In [8]:
#Pull the sauce list, ingredients, and ratings from hotsaucefever.com

sauceList = []
sauceIngredients = []
taste = []
aroma = []
looks = []
heat = []
label = []
for page in range(1,25):
    url="https://hotsaucefever.com/hotsauce/page/"+str(page)+"/?sort=hottestshu.desc"
    print(url)

# Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text

# Parse the html content
    soup = BeautifulSoup(html_content)
        
# Find all <h2> in the html
    h2 = soup.find_all('h2')
    
# For loop to get list of sauce names
    for text in h2:
        text = str(text)
        if "a href" in text:
            splittext = text.split(">")
            splittext = splittext[2].split("<")[0]
            sauceList.append(splittext)

# URL for each sauce to get ingredients list and ratings                     
            sauceURL = "https://hotsaucefever.com" + text.split("\"")[1]

# Get HTML
            html_content = requests.get(sauceURL).text
            soup = BeautifulSoup(html_content)
        
# Find all <div> in html and split out the ingredients list
            p = soup.find_all('div')
            p = str(p).split('Ingredients:')
            if len(p)>1:
                p = p[1].split('p>')
                ingredients = str(p[1]).replace("</","") #removes unnecessary text at end of ingredients list
                sauceIngredients.append(ingredients)
                              
            else:
                sauceIngredients.append('N/A')
                
            html_content = requests.get(sauceURL).text
            soup = BeautifulSoup(html_content)
            ul = soup.find_all('ul')
            if 'no reviews' in str(ul).lower():
                taste.append(0)
                aroma.append(0)
                looks.append(0)
                heat.append(0)
                label.append(0)
    
            else:
    
                taste.append(float((str(ul[5]).split('Taste:</em> ')[1])[0:3].split('</')[0]))
                aroma.append(float((str(ul[5]).split('Aroma:</em> ')[1])[0:3].split('</')[0]))
                looks.append(float((str(ul[5]).split('Looks:</em> ')[1])[0:3].split('</')[0]))
                heat.append(float((str(ul[5]).split('Heat:</em> ')[1])[0:3].split('</')[0]))
                label.append(float((str(ul[5]).split('Label:</em> ')[1])[0:3].split('</')[0]))

                
# Get SHU for each sauce
SHU = []
for page in range(1,3):
    url="https://hotsaucefever.com/hotsauce/page/"+str(page)+"/?sort=hottestshu.desc"
   

# Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text


# Parse the html content
    soup = BeautifulSoup(html_content)
    
# Find all <p> in the html
    span = soup.find_all('p')
    for i in range(1,26):
        SHU.append((str(span).split('SHU')[i].split('</span>')[2].split('<br/>')[0].strip()))

#For 3rd page
url="https://hotsaucefever.com/hotsauce/page/3/?sort=hottestshu.desc"

html_content = requests.get(url).text

soup = BeautifulSoup(html_content)
    
span = soup.find_all('p')
for i in range(1,13):
    SHU.append((str(span).split('SHU')[i].split('</span>')[2].split('<br/>')[0].strip()))
    
# Put into dataframe and parse ingredients list
sauceDF = pd.DataFrame()
sauceDF['Hot Sauce'] = sauceList
sauceDF['Ingredients'] = sauceIngredients

#SHU is currently a list of strings. Change to int

for i in range(len(SHU)):
    SHU[i] = int(SHU[i].replace(',',""))
    
for i in range(len(sauceDF['Hot Sauce'])-len(SHU)):
    SHU.append(0)

sauceDF['SHU'] = SHU
sauceDF['Taste'] = taste
sauceDF['Aroma'] = aroma
sauceDF['Looks'] = looks
sauceDF['Heat'] = heat
sauceDF['Label'] = label
sauceDF['Capsaicin?'] = np.where(sauceDF['Ingredients'].str.contains('Capsaicin'), 1, 0)
sauceDF['Habanero?'] = np.where(sauceDF['Ingredients'].str.contains('Habanero'), 1, 0)
sauceDF['Jalapeno?'] = np.where(sauceDF['Ingredients'].str.contains('Jalape'), 1, 0)
sauceDF['Ghost?'] = np.where(sauceDF['Ingredients'].str.contains('Jolokia') |sauceDF['Ingredients'].str.contains('Ghost'), 1, 0)
sauceDF['Scorpion?'] = np.where(sauceDF['Ingredients'].str.contains('Scorpion'), 1, 0)
sauceDF['Carolina Reaper?'] = np.where(sauceDF['Ingredients'].str.contains('Reaper'), 1, 0)
sauceDF['Cayenne?'] = np.where(sauceDF['Ingredients'].str.contains('Cayenne'), 1, 0)
sauceDF['Poblano?'] = np.where(sauceDF['Ingredients'].str.contains('Poblano'), 1, 0)
sauceDF['Serrano?'] = np.where(sauceDF['Ingredients'].str.contains('Serrano'), 1, 0)
sauceDF['Vinegar?'] = np.where(sauceDF['Ingredients'].str.contains('Vinegar'), 1, 0)
sauceDF['Water?'] = np.where(sauceDF['Ingredients'].str.contains('Water'), 1, 0)
sauceDF['Oil?'] = np.where(sauceDF['Ingredients'].str.contains('Oil')&
                             ~sauceDF['Ingredients'].str.contains('Chilli Oil')&
                             ~sauceDF['Ingredients'].str.contains('Capsium Oil')&
                             ~sauceDF['Ingredients'].str.contains('Lime Oil')&
                             ~sauceDF['Ingredients'].str.contains('Lemon Oil')&
                             ~sauceDF['Ingredients'].str.contains('Orange Oil'), 1, 0)
sauceDF['Fruit?'] = np.where(sauceDF['Ingredients'].str.contains('Mango')|
                             sauceDF['Ingredients'].str.contains('Pineapple')|
                             sauceDF['Ingredients'].str.contains('Oranges')|
                             sauceDF['Ingredients'].str.contains('Orange Juice')|
                            (sauceDF['Ingredients'].str.contains('Apple')& ~sauceDF['Ingredients'].str.contains('Vinegar'))|
                             sauceDF['Ingredients'].str.contains('Guava')|
                             sauceDF['Ingredients'].str.contains('Papaya')|
                             sauceDF['Ingredients'].str.contains('Apricot')|
                             sauceDF['Ingredients'].str.contains('Raisin')|
                             sauceDF['Ingredients'].str.contains('Pear')|
                             sauceDF['Ingredients'].str.contains('ruit')|
                             sauceDF['Ingredients'].str.contains('Kiwi')|
                             sauceDF['Ingredients'].str.contains('Cherr'), 1, 0)
sauceDF['Vegetable?'] = np.where(sauceDF['Ingredients'].str.contains('Carrot')|
                             sauceDF['Ingredients'].str.contains('Squash')|
                             sauceDF['Ingredients'].str.contains('Potato')|
                             sauceDF['Ingredients'].str.contains('Tomato')|
                             sauceDF['Ingredients'].str.contains('Onion')|
                             sauceDF['Ingredients'].str.contains('Scallion')|
                             sauceDF['Ingredients'].str.contains('Peas'), 1, 0)
sauceDF['Sweetener?'] = np.where(sauceDF['Ingredients'].str.contains('Sweetener')|
                             sauceDF['Ingredients'].str.contains('Sugar')|
                             sauceDF['Ingredients'].str.contains('Honey')|
                             sauceDF['Ingredients'].str.contains('Agave')|
                             sauceDF['Ingredients'].str.contains('Molasses')|
                             sauceDF['Ingredients'].str.contains('Syrup'), 1, 0)
sauceDF['Thickener?'] = np.where(sauceDF['Ingredients'].str.contains('Thickener')|
                             sauceDF['Ingredients'].str.contains('Gum'), 1, 0)


https://hotsaucefever.com/hotsauce/page/1/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/2/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/3/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/4/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/5/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/6/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/7/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/8/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/9/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/10/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/11/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/12/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/13/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/14/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce/page/15/?sort=hottestshu.desc
https://hotsaucefever.com/hotsauce

In [11]:
#Save as CSV
sauceDF.to_csv('sauce_dataframe.csv',index = False, header=True)

,Hot Sauce,Ingredients,SHU,Taste,Aroma,Looks,Heat,Label,Capsaicin?,Habanero?,...,Cayenne?,Poblano?,Serrano?,Vinegar?,Water?,Oil?,Fruit?,Vegetable?,Sweetener?,Thickener?
14,Manofuel - Inferno's Ghost Hot Sauce,"Vinegar, Cayenne Pepper Sauce, Extra Virgin Ol...",500000,4.9,4.9,4.9,5.0,4.9,0,1,...,1,0,0,1,0,1,0,0,0,0
20,Todd's Originals - Inner Beauty Hot Sauce,"Mustard, Distilled Vinegar, Scotch Bonnet Pepp...",150000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,1,1,0,1,0
28,Dieterich's - Carolina Reaper Sauce,"Fermented Cane Vinegar(water, Sugar Cane Molas...",70000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,1,0,1,1,1
30,Dieterich's - Ghost Pepper Sauce,"Fermented Cane Vinegar(water, Sugar Cane Molas...",50000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,1,0,1,1,1
40,Pantry by Nature - Tiger Sate: Hot,"Fresh Lemongrass, Garlic, Habanero Or Jalapeno...",5000,0.0,0.0,0.0,0.0,0.0,0,1,...,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,Sam &amp; Oliver - Stoke Chipotle Pepper Sauce,"Distilled Vinegar, Chipotle Peppers (Smoked Ja...",0,4.2,4.0,4.0,4.0,5.0,0,0,...,0,0,0,1,1,1,0,1,1,0
531,Bandar - Spicy Mango Chili Sauce,"Mango, Water, Salt, Green Chili, Mixed Spices ...",0,3.8,3.0,3.5,3.0,4.0,0,0,...,0,0,0,0,1,1,1,0,0,0
534,Granddaddy's Hot Sauce,"Tomato Concentrate, Apple Cider Vinegar, Vineg...",0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,1,1,1,1,0
547,Golden Toad - Xtinction Sauce Original Ghost P...,"Naga Jolokia, Red Habanero, Vinegar, Water, Sa...",0,4.9,4.8,5.0,4.6,4.2,0,1,...,0,0,0,1,1,1,0,0,1,0
